In [ ]:
#Complete Pipeline 

class CarbonCapturePipeline:

    def __init__(self, enterprise_id, is_msme=True):
        print("=" * 70)
        print("CARBON CAPTURE AI PIPELINE (WITH FLOW SENSOR)")
        print("=" * 70)

        self.enterprise_id = enterprise_id

        print("\n[1/4] Anomaly Detector...")
        self.anomaly_detector = CarbonCaptureAnomalyDetector()

        print("[2/4] RL Controller...")
        self.rl_controller = RLFanController()

        print("[3/4] Credit Calculator...")
        self.credit_calculator = CarbonCreditCalculator(fsme=1.2 if is_msme else 1.0)

        print("[4/4] Blockchain...")
        self.blockchain = BlockchainTokenizer()

        self.current_fan_speed = 1500
        self.is_trained = False

        print("\n Pipeline initialized!")
        print("=" * 70)

    def train_models(self, training_data):
        print("\n" + "=" * 70)
        print("TRAINING")
        print("=" * 70)

        if isinstance(training_data, str):
            df = pd.read_csv(training_data)
        else:
            df = training_data

        print(f"\nTraining data: {len(df)} records")

        # Training anomaly detector
        print("\n[1/2] Training Anomaly Detector...")
        features_list = []
        for _, row in df.iterrows():
            features = self.anomaly_detector.create_kinetic_fingerprint(
                row['CO2_ppm'],
                row['Temperature_C'],
                row['Humidity_Percent'],
                row['Gas_Flow_L_per_min'], 
                row['Weight_Change_g'],
                row['Fan_Speed_RPM']
            )
            features_list.append(features[0])

        features_array = np.array(features_list)
        self.anomaly_detector.train(features_array)

        # Training Reinforcment Learning Controller
        print("\n[2/2] Training RL Controller...")
        for episode in range(5):
            reward = self.rl_controller.train_episode(
                df['CO2_ppm'].values,
                df['Temperature_C'].values,
                df['Gas_Flow_L_per_min'].values, 
                df['Weight_Change_g'].values
            )
            print(f"  Episode {episode+1}/5 - Reward: {reward:.2f}")

        self.is_trained = True
        print("\n Training complete!")
        print("=" * 70)

    def process_single_record(self, record):
        # Step 1: RL Control
        if self.is_trained:
            new_fan_speed, _, _ = self.rl_controller.control_fan_speed(
                record['CO2_ppm'],
                record['Temperature_C'],
                self.current_fan_speed,
                record['Gas_Flow_L_per_min'], 
                explore=False
            )
            self.current_fan_speed = new_fan_speed
        else:
            new_fan_speed = record['Fan_Speed_RPM']

        # Step 2: Anomaly Detection 
        features = self.anomaly_detector.create_kinetic_fingerprint(
            record['CO2_ppm'],
            record['Temperature_C'],
            record['Humidity_Percent'],
            record['Gas_Flow_L_per_min'], 
            record['Weight_Change_g'],
            new_fan_speed
        )

        anomaly_result = self.anomaly_detector.detect_anomaly(features)
        fvalid = anomaly_result['fvalid']

        # Step 3: Credits 
        credit_result = self.credit_calculator.calculate_E_net(
            record['CO2_ppm'],
            record['Gas_Flow_L_per_min'], 
            record['Weight_Change_g'],
            fvalid
        )

        # Step 4: Tokens
        if credit_result['is_valid'] and credit_result['carbon_credits'] > 0:
            token_result = self.blockchain.mint_tokens(
                credit_result['carbon_credits'],
                self.enterprise_id,
                fvalid
            )
        else:
            token_result = {
                'success': False,
                'error': 'Invalid',
                'tokens_minted': 0,
                'total_value_inr': 0
            }

        return {
            'fan_speed': new_fan_speed,
            'anomaly': anomaly_result,
            'credits': credit_result,
            'tokens': token_result
        }

    def process_batch(self, data):
        print("\n" + "=" * 70)
        print("PROCESSING")
        print("=" * 70)

        if isinstance(data, str):
            df = pd.read_csv(data)
        else:
            df = data

        print(f"\nProcessing {len(df)} records...")

        results = []
        for idx, row in df.iterrows():
            result = self.process_single_record(row)

            results.append({
                'Record_ID': idx + 1,
                'Timestamp': row['Timestamp'],
                'CO2_ppm': row['CO2_ppm'],
                'Temperature_C': row['Temperature_C'],
                'Humidity_%': row['Humidity_Percent'],
                'Gas_Flow_L_min': row['Gas_Flow_L_per_min'],  # ← SHOW FLOW
                'Weight_g': row['Weight_Change_g'],
                'Fan_RPM': result['fan_speed'],
                'Is_Anomaly': result['anomaly']['is_anomaly'],
                'Fvalid': result['anomaly']['fvalid'],
                'E_net': result['credits']['E_net'],
                'Credits': result['credits']['carbon_credits'],
                'Status': result['credits']['status'],
                'Tokens': result['tokens'].get('tokens_minted', 0),
                'Value_INR': result['tokens'].get('total_value_inr', 0),
                'Value_ETH': result['tokens'].get('total_value_eth', 0)
            })

        results_df = pd.DataFrame(results)

        print(f"\n✓ Complete!")
        print("=" * 70)

        return results_df

    def generate_report(self):
        print("\n" + "=" * 70)
        print("REPORT")
        print("=" * 70)

        portfolio = self.blockchain.get_portfolio(self.enterprise_id)

        print(f"\nEnterprise: {self.enterprise_id}")
        print(f"\n Credits: {portfolio['carbon_credits']:.6f} tonnes CO2")
        print(f"\n🪙 CUBEX: {portfolio['cubex_balance']:.6f}")
        print(f"   Value: ₹{portfolio['value_inr']:,.2f}")
        print(f"   Value: {portfolio['value_eth']:.6f} ETH")

        print("=" * 70)

        return portfolio

print("Main Pipeline loaded")